<b><h3>Introduction/Business Problem</h3></b>
The problem is to find the best place in Moscow to open a restaurant. 
What is the place that has the most reviews, what location seems to be the most popular. 
Is there a type of  cafe that is more popular than another?

The problem will be solved using the Foursquare API, looking at the usercount and tipcount. This problem will help everyone that wants to open up a cafe in London and wants to know if it is a viable option where they intend on opening.

<h3>Data</h3>
The data that will be used is Foursquare data for London. It will focus on venues of cafe type. The popularity will be an indicator for how good an area is for a cafe.

In [2]:
!pip install beautifulsoup4
!pip install lxml
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 


from IPython.display import display_html
import pandas as pd
import numpy as np
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Folium installed')
print('Libraries imported.')

     |████████████████████████████████| 122kB 6.3MB/s eta 0:00:01
     |████████████████████████████████| 5.5MB 5.7MB/s eta 0:00:01     |██████████████▎                 | 2.5MB 5.7MB/s eta 0:00:01     |███████████████████▎            | 3.3MB 5.7MB/s eta 0:00:01
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.

In [3]:
# Get longitude and latitude for Moscow
address = 'London, London'

geolocator = Nominatim(user_agent="msk_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London is 51.5073219, -0.1276474.


In [4]:
# Set up Foursquare
CLIENT_ID = 'AHMRXMTIUDBMGLDM3GJ013J30ZBSYZAO1ULLDEOVO0JVVUI2' # your Foursquare ID
CLIENT_SECRET = 'E1GF05B43XDJXVE4TVFVL3PFG3HUCHWD0OBR5ZK3TQGFCZQS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AHMRXMTIUDBMGLDM3GJ013J30ZBSYZAO1ULLDEOVO0JVVUI2
CLIENT_SECRET:E1GF05B43XDJXVE4TVFVL3PFG3HUCHWD0OBR5ZK3TQGFCZQS


In [5]:
search_query = 'cafe' 
radius = 50000
print(search_query + ' .... OK!')

cafe .... OK!


In [6]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=AHMRXMTIUDBMGLDM3GJ013J30ZBSYZAO1ULLDEOVO0JVVUI2&client_secret=E1GF05B43XDJXVE4TVFVL3PFG3HUCHWD0OBR5ZK3TQGFCZQS&ll=51.5073219,-0.1276474&v=20180605&query=cafe&radius=50000&limit=500'

In [7]:
results = requests.get(url).json()

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  import sys


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.neighborhood,venuePage.id
0,4b228bfef964a520c34824e3,Café In The Crypt,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1590498524,False,St. Martin-in-the-Fields,51.508962,-0.126729,"[{'label': 'display', 'lat': 51.50896230356763...",193,WC2N 4JJ,GB,London,Greater London,United Kingdom,"[St. Martin-in-the-Fields, London, Greater Lon...",NaN,NaN,NaN
1,4af4407cf964a520e3f021e3,Cafe On The Square,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1590498524,False,Trafalgar Sq,51.508304,-0.127786,"[{'label': 'display', 'lat': 51.50830388767846...",109,WC2N 5DN,GB,London,Greater London,United Kingdom,"[Trafalgar Sq, London, Greater London, WC2N 5D...",NaN,NaN,NaN
2,58b58c0c11742034679eb5e3,St James's Café,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1590498524,False,St James's Park,51.503998,-0.131390,"[{'label': 'display', 'lat': 51.50399774156685...",451,SW1A 2BJ,GB,London,Greater London,United Kingdom,"[St James's Park, London, Greater London, SW1A...",NaN,NaN,NaN
3,4ba3cdf5f964a520606138e3,The National Café,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1590498524,False,Trafalgar Sq,51.508803,-0.127544,"[{'label': 'display', 'lat': 51.50880255694977...",164,WC2N 5DN,GB,London,Greater London,United Kingdom,"[Trafalgar Sq (St. Martin's Pl), London, Great...",St. Martin's Pl,NaN,NaN
4,5c1e4b2ed41bb7003995b2d6,Hard Rock Cafe Piccadilly Circus,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",v-1590498524,False,"Criterion Building, 225-229 Piccadilly",51.509974,-0.133663,"[{'label': 'display', 'lat': 51.509974, 'lng':...",510,W1J 9HR,GB,London,Greater London,United Kingdom,"[Criterion Building, 225-229 Piccadilly (Crite...",Criterion Building,NaN,NaN


In [8]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Café In The Crypt,Café,St. Martin-in-the-Fields,51.508962,-0.126729,"[{'label': 'display', 'lat': 51.50896230356763...",193,WC2N 4JJ,GB,London,Greater London,United Kingdom,"[St. Martin-in-the-Fields, London, Greater Lon...",NaN,NaN,4b228bfef964a520c34824e3
1,Cafe On The Square,Café,Trafalgar Sq,51.508304,-0.127786,"[{'label': 'display', 'lat': 51.50830388767846...",109,WC2N 5DN,GB,London,Greater London,United Kingdom,"[Trafalgar Sq, London, Greater London, WC2N 5D...",NaN,NaN,4af4407cf964a520e3f021e3
2,St James's Café,Café,St James's Park,51.503998,-0.131390,"[{'label': 'display', 'lat': 51.50399774156685...",451,SW1A 2BJ,GB,London,Greater London,United Kingdom,"[St James's Park, London, Greater London, SW1A...",NaN,NaN,58b58c0c11742034679eb5e3
3,The National Café,Café,Trafalgar Sq,51.508803,-0.127544,"[{'label': 'display', 'lat': 51.50880255694977...",164,WC2N 5DN,GB,London,Greater London,United Kingdom,"[Trafalgar Sq (St. Martin's Pl), London, Great...",St. Martin's Pl,NaN,4ba3cdf5f964a520606138e3
4,Hard Rock Cafe Piccadilly Circus,American Restaurant,"Criterion Building, 225-229 Piccadilly",51.509974,-0.133663,"[{'label': 'display', 'lat': 51.509974, 'lng':...",510,W1J 9HR,GB,London,Greater London,United Kingdom,"[Criterion Building, 225-229 Piccadilly (Crite...",Criterion Building,NaN,5c1e4b2ed41bb7003995b2d6


In [10]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker 
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='London',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the cafes as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map